**Set environment**

In [1]:
source ../run_config_project.sh
show_env

You are working on             Duke Server: HARDAC
BASE DIRECTORY (FD_BASE):      /data/reddylab/Kuei
REPO DIRECTORY (FD_REPO):      /data/reddylab/Kuei/repo
WORK DIRECTORY (FD_WORK):      /data/reddylab/Kuei/work
DATA DIRECTORY (FD_DATA):      /data/reddylab/Kuei/data
CONTAINER DIR. (FD_SING):      /data/reddylab/Kuei/container

You are working with           ENCODE FCC
PATH OF PROJECT (FD_PRJ):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC
PROJECT RESULTS (FD_RES):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results
PROJECT SCRIPTS (FD_EXE):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts
PROJECT DATA    (FD_DAT):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data
PROJECT NOTE    (FD_NBK):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks
PROJECT DOCS    (FD_DOC):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs
PROJECT LOG     (FD_LOG):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log
PROJECT APP     (FD_APP):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/app
PROJEC

## Prepare

**Set assay names**

In [2]:
### Assay
ASSAY_TYPE=assay_fcc
ASSAY_NAMES=(
    STARR_ATAC_K562_Reddy_KS274
    STARR_ATAC_K562_Reddy_KS91
    STARR_WHG_K562_Reddy_A001
)

In [3]:
ls -1 ${FD_RES}/assay_fcc

MPRA_Tiling_K562_Tewhey_hannah
STARR_ATAC_K562_Reddy_KS274
STARR_ATAC_K562_Reddy_KS91
STARR_WHG_K562_Reddy_A001


**Test loop**

In [5]:
for ASSAY_NAME in ${ASSAY_NAMES[@]}; do
    
    ### set log file
    FN_LOG=coverage.library_size.${ASSAY_NAME}.txt
    FP_LOG=${FD_LOG}/${FN_LOG}
    
    ### show progress
    echo ======================================
    echo ${ASSAY_NAME}
    echo "Total #{files} =" ${#FP_INPS[@]}
    echo "Log file: " '${FD_LOG}/'${FN_LOG}
    
    
    ### set input and output files
    FD_INP=${FD_RES}/assay_fcc/${ASSAY_NAME}/fragment_counts
    FP_INPS=($(ls ${FD_INP}/*WGS*bed.gz))
    
    for FP_INP in ${FP_INPS[@]}; do

        ### show I/O file
        echo --------------------------------------
        echo "Input: " ${FP_INP}
        echo
        echo "show first few lines of file"
        zcat ${FP_INP} | head -n 3
        echo
    done
    echo
done

STARR_ATAC_K562_Reddy_KS274
Total #{files} = 7
Log file:  ${FD_LOG}/coverage.library_size.STARR_ATAC_K562_Reddy_KS274.txt
--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS274.hg38.Output.rep1.WGS.unstranded.bed.gz

show first few lines of file
chr1	14145	14485	chr1:14145-14485	2	.
chr1	14243	14498	chr1:14243-14498	1	.
chr1	14454	14729	chr1:14454-14729	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS274.hg38.Output.rep2.WGS.unstranded.bed.gz

show first few lines of file
chr1	13055	13163	chr1:13055-13163	1	.
chr1	13236	13481	chr1:13236-13481	1	.
chr1	13253	13495	chr1:13253-13495	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS2

## Execute

**Run: Calculate library size for each sample in each assay**

In [6]:
for ASSAY_NAME in ${ASSAY_NAMES[@]}; do
    
    ### set log file
    FN_LOG=coverage.library_size.${ASSAY_NAME}.txt
    FP_LOG=${FD_LOG}/${FN_LOG}
    
    ### show progress
    echo ======================================
    echo ${ASSAY_NAME}
    echo "Total #{files} =" ${#FP_INPS[@]}
    echo "Log file: " '${FD_LOG}/'${FN_LOG}
    
    sbatch -p ${NODE} \
        --exclude=dl-01   \
        --cpus-per-task 4 \
        --mem 4G \
        --output ${FP_LOG} \
        --export=FD_PRJ=${FD_PRJ},ASSAY_NAME=${ASSAY_NAME} \
        <<'EOF'
#!/bin/bash
### set environment
source ${FD_PRJ}/scripts/config_project.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/assay_fcc/${ASSAY_NAME}/fragment_counts
FP_INPS=($(ls ${FD_INP}/*WGS*bed.gz))

FD_OUT=${FD_RES}/assay_fcc/${ASSAY_NAME}/fragment_counts/summary
FN_OUT=result.library_size.csv
FP_OUT=${FD_OUT}/${FN_OUT}

### init
mkdir -p   ${FD_OUT}
echo  -n > ${FP_OUT}

### execute
###     loop through each file and get total fragment counts
for FP_INP in ${FP_INPS[@]}; do

    ### show I/O file
    echo --------------------------------------
    echo "Input: " ${FP_INP}
    echo
    echo "show first few lines of file"
    fun_cat ${FP_INP} | head -n 3
    echo
    
    ### count the total counts and append the results to the output file
    FN_INP=$(basename ${FP_INP})
    zcat ${FP_INP} |\
        awk -v OFS=, -v FNAME=${FN_INP} '{sum += $5} END {print FNAME, sum}' \
        >> ${FP_OUT}
done

### show I/O file
echo "Output file: ${FP_OUT}"
echo
echo "show lines of file"
fun_cat ${FP_OUT}
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF
    echo
done

STARR_ATAC_K562_Reddy_KS274
Total #{files} = 7
Log file:  ${FD_LOG}/coverage.library_size.STARR_ATAC_K562_Reddy_KS274.txt
Submitted batch job 30733971

STARR_ATAC_K562_Reddy_KS91
Total #{files} = 7
Log file:  ${FD_LOG}/coverage.library_size.STARR_ATAC_K562_Reddy_KS91.txt
Submitted batch job 30733972

STARR_WHG_K562_Reddy_A001
Total #{files} = 7
Log file:  ${FD_LOG}/coverage.library_size.STARR_WHG_K562_Reddy_A001.txt
Submitted batch job 30733973



## Review

**Check results**

In [13]:
cat ${FD_LOG}/coverage.library_size.STARR_ATAC_K562_Reddy_KS274.txt

Hostname:           x1-01-2.genome.duke.edu
Slurm Array Index: 
Time Stamp:         04-03-24+09:56:54

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS274.hg38.Output.rep1.WGS.unstranded.bed.gz

show first few lines of file
chr1	14145	14485	chr1:14145-14485	2	.
chr1	14243	14498	chr1:14243-14498	1	.
chr1	14454	14729	chr1:14454-14729	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS274.hg38.Output.rep2.WGS.unstranded.bed.gz

show first few lines of file
chr1	13055	13163	chr1:13055-13163	1	.
chr1	13236	13481	chr1:13236-13481	1	.
chr1	13253	13495	chr1:13253-13495	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS274/fragment_counts/ASTARRseq_K562_KS274.hg38.Output.rep3

In [14]:
cat ${FD_LOG}/coverage.library_size.STARR_ATAC_K562_Reddy_KS91.txt

Hostname:           x1-01-2.genome.duke.edu
Slurm Array Index: 
Time Stamp:         04-03-24+09:56:53

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/fragment_counts/ASTARRseq_K562_KS91.hg38.Input.rep1.WGS.unstranded.bed.gz

show first few lines of file
chr1	10001	10143	chr1_10001_10143	1	.
chr1	10002	10118	chr1_10002_10118	1	.
chr1	10002	10131	chr1_10002_10131	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/fragment_counts/ASTARRseq_K562_KS91.hg38.Input.rep2.WGS.unstranded.bed.gz

show first few lines of file
chr1	10001	10233	chr1_10001_10233	1	.
chr1	10001	10351	chr1_10001_10351	1	.
chr1	10002	10119	chr1_10002_10119	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/fragment_counts/ASTARRseq_K562_KS91.hg38.Input.rep3.WGS.unst

In [15]:
cat ${FD_LOG}/coverage.library_size.STARR_WHG_K562_Reddy_A001.txt

Hostname:           x1-01-2.genome.duke.edu
Slurm Array Index: 
Time Stamp:         04-03-24+09:56:54

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/fragment_counts/WSTARRseq_K562_A001.hg38.Input.rep1.WGS.unstranded.bed.gz

show first few lines of file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/fragment_counts/WSTARRseq_K562_A001.hg38.Input.rep2.WGS.unstranded.bed.gz

show first few lines of file
chr1	10244	10370	chr1_10244_10370	1	.
chr1	14494	14961	chr1_14494_14961	1	.
chr1	15593	16258	chr1_15593_16258	1	.

--------------------------------------
Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/fragment_counts/WSTARRseq_K562_A001.hg38.Input.rep3.WGS.unstran

In [11]:
ls /data/reddylab/Alex/encode4_duke/data/starr_seq

A001-input-K562-rep1.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep2.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep3.masked.dedup.fragments.counts.txt.gz
A001-input-K562-rep4.masked.dedup.fragments.counts.txt.gz
A001-K562-rep1.masked.dedup.fragments.bedpe.gz
A001-K562-rep1.masked.dedup.fragments.counts.txt.gz
A001-K562-rep2.masked.dedup.fragments.bedpe.gz
A001-K562-rep2.masked.dedup.fragments.counts.txt.gz
A001-K562-rep3.masked.dedup.fragments.bedpe.gz
A001-K562-rep3.masked.dedup.fragments.counts.txt.gz


In [12]:
ls /data/reddylab/Alex/encode4_duke/data

atac_seq       chip_seq           rna_seq              starr_seq
atacstarr_seq  data_distribution  sharing_within_duke
